In [1]:
import pandas as pd
import numpy as np
import time
import json
import requests
from datetime import date
from matplotlib import pyplot as plt
import datetime
import timeit

In [2]:
# ADD API KEYS HERE - they will all be used in API calls as to avoid rate limiting
api_keys = ['31bf3a0ba5585b5f5de37aedef10ffcd',
            '855d1f6526f52f62fc4aeec74354a62e',
            '7de8bd3327159f7385e23508b0b71419',
            '5a767fa6f202e0f08d366232f99e8d3c',
            'ef64740368755f761543d05127051589',
            'af06c12c93dd48337853ff2daf61e8ef']

In [3]:
state_level_feat = pd.read_csv('state_features_final.csv')

In [4]:
chosen_state_feat = np.unique(state_level_feat.title.values)
print(np.unique(state_level_feat['frequency']))
print(len(chosen_state_feat))
# print(chosen_state_feat)

['Annual' 'Monthly' 'Quarterly' 'Quarterly, End of Period'
 'Weekly, Ending Saturday']
181


In [5]:
state_level_feat.head()
# state_list = np.unique(state_level_feat['state_id'].values)
# print(state_list)
# to_retry = np.unique([27300])

,Unnamed: 0,id,title,observation_start,observation_end,frequency,units,seasonal_adjustment,state_id
0,3,AGEXMAL1A647NCEN,Age 65 and Over Tax Exemptions for,1989-01-01,2017-01-01,Annual,Number of Exemptions,Not Seasonally Adjusted,27282
1,45,ALALLL,Allowance for Loan and Lease Losses for Commer...,1984-01-01,2019-10-01,Quarterly,Thousands of Dollars,Not Seasonally Adjusted,27282
2,77,ALATRR,Allocated Transfer Risk Reserves for Commercia...,1984-01-01,2019-10-01,Quarterly,Thousands of Dollars,Not Seasonally Adjusted,27282
3,110,ALBP1FHSA,New Private Housing Units Authorized by Buildi...,1988-01-01,2020-02-01,Monthly,Units,Seasonally Adjusted,27282
4,112,ALBPPRIVSA,New Private Housing Units Authorized by Buildi...,1988-01-01,2020-02-01,Monthly,Units,Seasonally Adjusted,27282


In [7]:
# edit the start and end variables to determine which counties to get data for
start = 0
end = 1
count = start

state_list = np.unique(state_level_feat['state_id'].values)
to_retry = ['27326']
feat_ids = state_level_feat['id'].values
cur_key_num = 0
num_keys = len(api_keys)
failed_ids = []
for state_id in to_retry:
    start_time = timeit.default_timer()
#     try:
    time.sleep(.1)
    params = {
        'category_id': state_id,
        'api_key': api_keys[cur_key_num],
        'file_type': 'json'
    }
    cur_key_num = (cur_key_num + 1) % num_keys

    r = requests.get(url = 'https://api.stlouisfed.org/fred/category/series', params=params)
    res_1000 = r.json()
    print(len(res_1000['seriess']))

    time.sleep(.1)
    params = {
        'category_id': state_id,
        'api_key': api_keys[cur_key_num],
        'file_type': 'json',
        'offset': 1000
    }
    cur_key_num = (cur_key_num + 1) % num_keys

    r2 = requests.get(url = 'https://api.stlouisfed.org/fred/category/series', params=params)
    res_2000 = r2.json()
    print(len(res_2000['seriess']))
    
    params = {
        'category_id': state_id,
        'api_key': api_keys[cur_key_num],
        'file_type': 'json',
        'offset': 2000
    }
    cur_key_num = (cur_key_num + 1) % num_keys
    
    r3 = requests.get(url = 'https://api.stlouisfed.org/fred/category/series', params=params)
    res_3000 = r3.json()
    print(len(res_3000['seriess']))
    
    params = {
        'category_id': state_id,
        'api_key': api_keys[cur_key_num],
        'file_type': 'json',
        'offset': 3000
    }
    cur_key_num = (cur_key_num + 1) % num_keys
    
    r4 = requests.get(url = 'https://api.stlouisfed.org/fred/category/series', params=params)
    res_4000 = r4.json()
    print(len(res_4000['seriess']))

    # Get the observations for a series
    seriess_1000 = res_1000['seriess']
    seriess_2000 = res_2000['seriess']
    seriess_3000 = res_3000['seriess']
    seriess_4000 = res_4000['seriess']
    seriess = seriess_1000 + seriess_2000 + seriess_3000 + seriess_4000

    res2 = []
    for series in range(len(seriess)):
        current_series = seriess[series]
        series_id = current_series['id']
        freq = current_series['frequency']

        if (freq != "Weekly, Ending Saturday"):
            params = {
                'series_id': series_id,
                'api_key': api_keys[cur_key_num],
                'file_type': 'json'
            }
        else:
            params = {
                'series_id': series_id,
                'api_key': api_keys[cur_key_num],
                'file_type': 'json',
                'frequency': 'm',
                'aggregation_method': 'avg'
            }
        cur_key_num = (cur_key_num + 1) % num_keys
        time.sleep(.1)

        if series_id in feat_ids:
            r = requests.get(url = 'https://api.stlouisfed.org/fred/series/observations', params=params)

            res1 = r.json()
            res1["observations"] = [i for i in res1["observations"] if not (int(i["date"][0:4]) < 2010 or int(i["date"][0:4]) > 2017)]

            for val in res1["observations"]:
                del val["realtime_start"]
                del val["realtime_end"]


            for val in res1['observations']:
                if (freq == "Annual"):
                    val['feature'] = series_id
                    res2 += [val]
                    for i in range(2, 13):
                        if i < 10:
                            string = '0' + str(i)
                        else:
                            string = str(i)
                        dict_to_add = {}
                        dict_to_add['feature'] = series_id
                        dict_to_add['date'] = val['date'][0:5] + string + val['date'][7:10]
                        dict_to_add['value'] = val['value']
                        res2 += [dict_to_add]
                elif (freq == "Quarterly" or freq == "Quarterly, End of Period"):
                    val['feature'] = series_id
                    res2 += [val]
                    for i in range(1, 3):
                        cur_date = int(val['date'][5:7])
                        date = i + cur_date
                        if date < 10:
                            date = '0' + str(date)
                        else:
                            date = str(date)

                        dict_to_add = {}
                        dict_to_add['feature'] = series_id
                        dict_to_add['date'] = val['date'][0:5] + date + val['date'][7:10]
                        dict_to_add['value'] = val['value']
                        res2 += [dict_to_add]
                elif (freq == "Weekly, Ending Saturday"):
                    val['feature'] = series_id
                    res2 += [val]
                else:
                    val['feature'] = series_id
                    res2 += [val]
    dates = []
    for i in range(96):
        dates += [res2[i]['date']]

    res3 = {}
    for val in res2:
        if val['feature'] not in res3.keys():
            # new thing
            res3[val['feature']] = [val['value']]
        else:
            res3[val['feature']] += [val['value']]

    for feat in res3:
        if len(res3[feat]) != 96:
            del res3[feat]

    name = str(state_id) + '.csv'

    data = pd.DataFrame(res3, index=dates)
    data.to_csv(name)
    elapsed = timeit.default_timer() - start_time
    print('%d. csv written, state id: %s \t time elapsed: %f' %(count, state_id, elapsed))
    count += 1

#     except:
#         print('State ID Failed, rate limited:', state_id)
#         failed_ids += [state_id]

print('Failed counties:')
if (len(failed_ids)) == 0:
    print("None")
else:
    for i in failed_ids:
        print(i)

1000
1000
1000
469
0. csv written, state id: 27326 	 time elapsed: 429.180900
Failed counties:
None


In [9]:
state_level_feat.head()
title_dict = {}
for index, row in state_level_feat.iterrows():
    title_dict[row["id"]] = row['title']
# for item in title_dict:
#     print(item, title_dict[item])


In [10]:
# code to rename the columns in the csvs
import os
# csvs = os.listdir('state_data')
csvs = ['27326.csv']

start_time = timeit.default_timer()

# print(csvs)
for idx, file in enumerate(csvs):
    df = pd.read_csv(file, index_col=0)
    df.rename(columns=title_dict, inplace=True)
    df = df.reindex(sorted(df.columns), axis=1)
    df.to_csv(('state_data/' + file))
    if (idx + 1) % 100 == 0:
        print(idx+1)
elapsed = timeit.default_timer() - start_time

In [3]:
# get the zillow data, and get the county names into the desired format (county_name County, state_abbr) (e.g. 'Fulton County, GA')
zillow = pd.read_csv('State_Zhvi_AllHomes.csv')
fred_state = pd.read_csv('state_table.csv')
zillow.insert(2, 'county_state', zillow['RegionName'])

# add a spot in the zillow dataframe to get the fred county IDs
zillow.insert(0, 'fred_state_id', 0)
# print(zillow)
# put the fred county IDs in the zillow dataframe
zillow['fred_state_id'] = [fred_state.loc[fred_state['name'] == state, 'id'].item() for state in zillow['county_state']]
zillow.head()

,fred_state_id,RegionID,SizeRank,county_state,RegionName,RegionType,StateName,1996-01-31,1996-02-29,1996-03-31,...,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31
0,27286,9,0,California,California,State,CA,162238.0,162008.0,161859.0,...,555305.0,556199.0,557569.0,559490.0,561495.0,563523.0,566433.0,569813.0,573975.0,578267.0
1,27326,54,1,Texas,Texas,State,TX,97631.0,97656.0,97684.0,...,206423.0,207079.0,207699.0,208380.0,208865.0,209358.0,209723.0,210164.0,210591.0,211199.0
2,27314,43,2,New York,New York,State,NY,139108.0,138793.0,138607.0,...,323518.0,324039.0,324498.0,324903.0,325371.0,325925.0,326646.0,327360.0,328101.0,328677.0
3,27291,14,3,Florida,Florida,State,FL,100595.0,100691.0,100803.0,...,244754.0,245385.0,245996.0,246632.0,247345.0,248103.0,248825.0,249766.0,250981.0,252309.0
4,150,21,4,Illinois,Illinois,State,IL,139112.0,138861.0,138544.0,...,203158.0,203458.0,203678.0,203788.0,203850.0,203907.0,204058.0,204302.0,204527.0,204839.0


In [4]:
# drop all columns from the zillow dataset that aren't fred_county_id or zillow price values
zillow.drop(columns=['RegionID', 'SizeRank', 'county_state',
                     'RegionName', 'RegionType', 'StateName'], inplace=True)
columns_list = list(zillow.columns)
columns_list.remove('fred_state_id')

# drop zillow price values outside the date range we care about
zillow.drop(columns=[column for column in columns_list if int(column[0:4]) < 2010 or int(column[0:4]) > 2017],
            inplace=True)

# make the fred_county_ids the indices for the dataframe
fred_state_id = zillow['fred_state_id']
zillow.drop(columns='fred_state_id', inplace=True)
zillow.set_index(fred_state_id, inplace=True)
# transpose the dataframe to get the dates as the index column, just like in the CSVs we got
zillow = zillow.transpose()

zillow.head()

fred_state_id,27286,27326,27314,27291,150,27320,27317,27304,27292,27315,...,27311,27322,27308,27289,27324,27283,27316,27329,27290,27334
2010-01-31,334006.0,137726.0,265765.0,159692.0,185760.0,168807.0,119593.0,104903.0,142475.0,162037.0,...,223434.0,248122.0,206470.0,242234.0,148145.0,263077.0,167874.0,226118.0,417998.0,197245.0
2010-02-28,333810.0,137702.0,265719.0,158933.0,185496.0,169004.0,119674.0,104532.0,141845.0,161572.0,...,222824.0,248750.0,206082.0,241984.0,148226.0,262922.0,167849.0,225943.0,417549.0,197087.0
2010-03-31,334587.0,137715.0,265568.0,158424.0,184139.0,169194.0,119611.0,104315.0,141244.0,161171.0,...,222164.0,249611.0,205988.0,241738.0,148361.0,262909.0,167536.0,226084.0,416565.0,197001.0
2010-04-30,335764.0,137807.0,265747.0,157944.0,184013.0,169324.0,119542.0,104180.0,141000.0,160933.0,...,221616.0,249226.0,205926.0,241213.0,148639.0,263137.0,167321.0,226230.0,418269.0,197123.0
2010-05-31,337485.0,137833.0,265827.0,157403.0,182880.0,169379.0,119363.0,104202.0,140817.0,160800.0,...,221479.0,248849.0,206039.0,240864.0,148940.0,263813.0,167575.0,226370.0,422245.0,197398.0


In [6]:
# print(np.unique(zillow[29611.0].values))
# print(zillow[29611.0].isnull().sum())
count2 = 0
for col in zillow.columns:
    val = zillow[col].isnull().sum()
    if val != 0:
        print("county: ", end = '')
        print(col, end = '')
        print(" = ", end = '')
        print(val)
        count2 = count2 + 1
print('States with NaN: ', end = '')
print(count2)
# for col in zillow.columns: 
#     print(col)

States with NaN: 0


In [ ]:
# For each csv in the county_data directory, add the zillow price index to the end of the csv.
# Save this with a similar file name in the county_data_zillow directory. If something goes wrong,
# print the name of the failed file.
import os
csvs = os.listdir('county_data')

count2 = 0
count3 = 0
start_time = timeit.default_timer()
for idx, file in enumerate(csvs):
    try:
        df = pd.read_csv(('county_data/' + file), index_col=0)
        df.head()
        df['Zillow Price Index'] = list(zillow[int(file[:-4])])
        val = df['Zillow Price Index'].isnull().sum()
        if val != 0:
            print("county: ", end = '')
            print(col, end = '')
            print(" = ", end = '')
            print(val)
            count2 = count2 + 1
        if val < 10 and val > 0:
            print("county: ", end = '')
            print(col, end = '')
            print(" = ", end = '')
            print(val)
            print(df['Zillow Price Index'])
            count3 = count3 + 1
#         df.to_csv(('county_data_zillow/' + file[:-4] + '_zillow.csv'))
    except:
        print('file {} failed'.format(file))
    if (idx + 1) % 100 == 0:
        print(idx+1)
elapsed = timeit.default_timer() - start_time
print(elapsed)


In [11]:
import os
# csvs = os.listdir('state_data')
csvs = ['27326.csv']

start_time = timeit.default_timer()
# csvs = ['27326.csv']
# file = csvs[0]
# df = pd.read_csv(('state_data/' + file), index_col=0)
# df.head()
# df.drop(columns='All-Transactions House Price Index for', inplace=True)
# for col in df.columns:
#     print(col)
# print(df.columns)

for idx, file in enumerate(csvs):
    try:
        df = pd.read_csv(('state_data/' + file), index_col=0)
    #         print(df.head())
        df.drop(columns='All-Transactions House Price Index for', inplace=True)
        df.to_csv(('state_data_zillow/' + file[:-4] + '_zillow.csv'))
        print("success")
    except:
        print('file {} failed'.format(file))
elapsed = timeit.default_timer() - start_time
# print(elapsed)

success
